In [1]:
from view_rides import add_path, show_path
import pandas as pd
import sys
sys.path.extend(['..', '../..'])
import psycopg2
from postgis.psycopg import register
from db_importer.settings import *
from adsp.db_utils import build_and_execute_query, getRideLocation
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib as mpl
from matplotlib import ticker
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import contextily as cx

class DatabaseConnection(object):
    def __enter__(self):
        self.conn = psycopg2.connect(f"dbname='{DB_NAME}' user='{DB_USER}' password='{DB_PASSWORD}' host='{DB_HOST}' port='{DB_PORT}'")
        self.conn.autocommit = True

        register(self.conn)
        self.cur = self.conn.cursor()

        return self.cur

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_tb is None:
            self.conn.commit()
            self.cur.close()
            self.conn.close()
        else:
            self.conn.rollback()
            self.cur.close()
            self.conn.close()

In [2]:
#filename = 'VM2_-186381149'

In [2]:
def get_sumo(file):
    #files = ['../parameter_evaluation/sim_data/oranienstr_new_params_all.csv', 'sim_data/oranienstr_new_params_slow.csv', 'sim_data/oranienstr_new_params_medium.csv', 'sim_data/oranienstr_new_params_fast.csv', 'sim_data/oranienstr_default.csv',]

    max_velos_arr = []

    df = pd.read_csv(file, delimiter=';')
    df = df[df.vehicle_id.str.startswith('f', na=False)]
    df['timestep_time'] *= 1e9
    #grouped = df.groupby('vehicle_id', group_keys=False)
    return df.rename(columns={'timestep_time':'timestamp', 'vehicle_id':'filename', 'vehicle_speed':'velo', 'vehicle_x':'lon','vehicle_y':'lat'})

In [3]:
def get_dataframe(start, end, exclude = (0, 0, 0, 0)):
    res = getRideLocation(start, end, exclude, time_diff="500")
    df = pd.DataFrame(res, columns=['filename', 'duration', 'timestamp', 'velo', 'coords', 'bike'])

    df['rad'] = 1

    df = df[~(df.rad.isnull() | df.timestamp.isnull() | df.coords.isnull())]

    df['lon'], df['lat'] = zip(*df.coords.values)

    return df

In [65]:
def analyze_scenario(df):

    #plt.xlim(0, 200)

    df2 = df[~df.velo.isna()]

    xmin = df2['lon'].min()
    xmax = df2['lon'].max()
    ymin = df2['lat'].min()
    ymax = df2['lat'].max()

    df_grouped = df2.groupby('filename', sort=False)

    def interact_plot(val):
        fig, ax = plt.subplots(1, 2, figsize=[30, 10])
        #ax = plt.axes()
        ax[0].grid()
        ax[0].xaxis.set_major_locator(ticker.LinearLocator(numticks=20))
        ax[0].set_ylabel('Speed (Km/h)')
        ax[0].set_xlabel('Time (timestamp / 1e9 -> seconds)')
        for ride_group_name in list(df_grouped.groups.keys())[:val]:
            df_ride_group = df_grouped.get_group(ride_group_name)
            points = np.array([df_ride_group['timestamp'].map(lambda v: (v - df_ride_group['timestamp'].iloc[0]) / 1e9), df_ride_group['velo'] * 3.6]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            lc = LineCollection(segments, cmap='copper', norm = plt.Normalize(0, 30))
            lc.set_array(df_ride_group['velo'] * 3.6)
            lc.set_linewidth(df_ride_group['velo']* 3.6 / 10 + 1)
            ax[0].add_collection(lc)
            ax[0].autoscale()
            #ax[0].plot(df_ride_group['timestamp'].map(lambda v: (v.value - df_ride_group['timestamp'].iloc[0].value) / 1e9), df_ride_group['velo'] * 3.6, label="One ride")
            points = np.array([df_ride_group['lon'], df_ride_group['lat']]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            lc = LineCollection(segments, cmap='copper', norm = plt.Normalize(0, 30))
            lc.set_array(df_ride_group['velo'] * 3.6)
            lc.set_linewidth(df_ride_group['velo']* 3.6 / 10 + 1)
            ax[1].add_collection(lc)
            ax[1].autoscale()
            ax[0].set_ylim(bottom=0, top=30)
            ax[0].set_xlim(left=0, right=450)
            #ax[1].plot(df_ride_group['lon'], df_ride_group['lat'])
        ax[1].set_xlim(xmin = xmin - 0.001, xmax = xmax + 0.001)
        ax[1].set_ylim(ymin = ymin - 0.001, ymax = ymax + 0.001)
        cx.add_basemap(ax[1], crs='EPSG:4326', source=cx.providers.Esri.WorldImagery)
        ax[1].set_aspect(1.7)

    interact(interact_plot, val=(1,len(df_grouped),1))

In [5]:
df = get_dataframe((13.416169,52.521508,13.416513,52.521832), (13.41591,52.522275,13.416421,52.522569), (13.415467,52.521919,13.416219,52.522107))

In [ ]:
df = get_dataframe((11.55562,48.14897,11.56095,48.14509), (11.55562,48.14897,11.56095,48.14509))

In [ ]:
df = get_dataframe((11.5570192378,48.1508535494,11.5581296723,48.1515603946), (11.5470709245,48.1589342674,11.5481813591,48.1596410013))

In [ ]:
df = get_dataframe((11.5470709245,48.1589342674,11.5481813591,48.1596410013), (11.5570192378,48.1508535494,11.5581296723,48.1515603946))

In [4]:
df = get_dataframe((11.067365735,49.4470913205,11.0686722994,49.4476597962), (11.079800456,49.446282173,11.0811070204,49.446850658))

In [21]:
df = get_dataframe((11.079800456,49.446282173,11.0811070204,49.446850658), (11.067365735,49.4470913205,11.0686722994,49.4476597962))

In [14]:
df = get_dataframe((9.7665499269,52.3569338146,9.7695218145,52.3594499164), (9.7447167455,52.3531397341,9.7476886331,52.3556560518))

In [12]:
df = get_dataframe((9.7447167455,52.3531397341,9.7476886331,52.3556560518), (9.7665499269,52.3569338146,9.7695218145,52.3594499164))

In [4]:
df = get_dataframe((9.720901382,52.3772149968,9.7238732696,52.3797299437), (9.7292698742,52.3694334729,9.7322417617,52.3719488629))

In [6]:
df = get_dataframe((9.7292698742,52.3694334729,9.7322417617,52.3719488629), (9.720901382,52.3772149968,9.7238732696,52.3797299437))

In [8]:
df = get_dataframe((9.7332093628,52.3788438556,9.7361812504,52.3813587096), (9.7237679871,52.3756083899,9.7267398747,52.3781234282))

In [10]:
df = get_dataframe((9.7237679871,52.3756083899,9.7267398747,52.3781234282), (9.7332093628,52.3788438556,9.7361812504,52.3813587096))

In [12]:
df = get_dataframe((11.059355754,49.4492168083,11.0606623184,49.4497852593), (11.0453386092,49.4538360222,11.0466451735,49.4544044196))

In [14]:
df = get_dataframe((11.0453386092,49.4538360222,11.0466451735,49.4544044196), (11.059355754,49.4492168083,11.0606623184,49.4497852593))

In [7]:
df = get_dataframe((11.079457580593482,49.44649236041058,11.080019894093809,49.446769576148), (11.06712002269169,49.44740684578576,11.068310105562006,49.44763829209129))

In [63]:
df_s = get_sumo('../parameter_evaluation/sim_data/frauentorgraben_new_params_all.csv')

In [64]:
analyze_scenario(df_s)

interactive(children=(IntSlider(value=50, description='val', min=1), Output()), _dom_classes=('widget-interact…

In [66]:
analyze_scenario(df)

interactive(children=(IntSlider(value=29, description='val', max=58, min=1), Output()), _dom_classes=('widget-…

In [ ]:
with DatabaseConnection() as con:
    cur = con.execute("""

    SELECT filename, unnest(timestamps) ts, (ST_DumpPoints(geom::geometry)).geom::json -> 'coordinates' as coords FROM ride WHERE st_intersects(geom,
            st_setsrid(st_makebox2d(st_makepoint(11.5595377784,48.1469343773), st_makepoint(11.5606482129,48.1476412765)), 4326))

    """)
    df = pd.DataFrame(con.fetchall(), columns=['filename', 'timestamp', 'coords'])
    df = df[~(df.timestamp.isnull() | df.coords.isnull())]
    df['lon'], df['lat'] = zip(*df.coords.values)
    df['rad'] = 1

df_grouped = df.groupby('filename', sort=False)
for i, ride_group_name in enumerate(df_grouped.groups):
    df_ride_group = df_grouped.get_group(ride_group_name)
    add_path(df_ride_group, lod=1)

show_path()

Your map should have been opened in your browser automatically.
Press ctrl+c to return.


In [3]:
with DatabaseConnection() as con:
    cur = con.execute("""

    SELECT filename, json_array_elements(coords) FROM (
    SELECT filename, ST_SimplifyPreserveTopology(geom::geometry, 0.01)::json -> 'coordinates' as coords FROM ride
    ) one

    """)
    df = pd.DataFrame(con.fetchall(), columns=['filename', 'coords'])
    df['timestamp'] = 0
    df = df[~(df.timestamp.isnull() | df.coords.isnull())]
    df['lon'], df['lat'] = zip(*df.coords.values)
    df['rad'] = 1

df_grouped = df.groupby('filename', sort=False)
for i, ride_group_name in enumerate(df_grouped.groups):
    df_ride_group = df_grouped.get_group(ride_group_name)
    add_path(df_ride_group, lod=1)

show_path()

Your map should have been opened in your browser automatically.
Press ctrl+c to return.


In [55]:
import cudf as cu
import cupy as cp
import cuml as cml

with DatabaseConnection() as con:
    cur = con.execute("""

    SELECT filename, timestamp, ((coords).geom::json -> 'coordinates' -> 0), ((coords).geom::json -> 'coordinates' -> 1) FROM unnested_ride LIMIT 1200000

    """)
    #arrx = con.fetchall()
    df = cu.DataFrame(con.fetchall(), columns=['filename', 'timestamp', 'lon', 'lat'])
    df = df[~(df.timestamp.isnull() | df.lon.isnull() | df.lat.isnull())]
    
    #df['lon'], df['lat'] = zip(*df.coords.values)
    #df['rad'] = 1

#cu_df = cu.DataFrame(df['coords'].values)

#dscan = cml.DBSCAN(eps=0.0005, min_samples=0, max_mbytes_per_batch=8192)
#dscan = cml.KernelDensity()
dscan = cml.AgglomerativeClustering(n_clusters=1000, n_neighbors=30)
#re = dscan.fit_predict(df[['lon', 'lat']])
#dscan.fit(df[['lon', 'lat']])
re = dscan.fit_predict(df[['lon', 'lat']])
#print(dscan.score_samples(cp.array([[8.0, 50.0]])))
ag = re.groupby(re).count()
print(ag)

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /project/include/rmm/mr/device/cuda_memory_resource.hpp:70: cudaErrorMemoryAllocation out of memory